In [1]:
import pandas as pd
import numpy as np
import re


# import data

In [2]:
original_input = pd.read_csv('input.txt',header=None, sep=' ', engine='python').applymap(lambda x: ''.join(sorted(x)))
ten_digits = original_input.loc[:,:9]
ten_digits

,0,1,2,3,4,5,6,7,8,9
0,abdefg,abcdfg,cd,bcdeg,bcdefg,abcdefg,cdef,bcd,bdefg,abceg
1,abcdefg,df,bcdfg,bdef,cdf,bcdefg,bcdeg,acdefg,abcdeg,abcfg
2,ag,afg,abcefg,abeg,acefg,abcdefg,abcdfg,acdef,bcdefg,bcefg
3,acdefg,bde,abcdefg,abcdg,abdeg,abcdef,adefg,befg,abdefg,be
4,bdefg,ab,acdef,abdefg,abf,abeg,abcdfg,abdef,abcdefg,bcdefg
...,...,...,...,...,...,...,...,...,...,...
195,abcdefg,bdefg,bcdef,efg,abcdef,eg,bceg,bcdefg,acdefg,abdfg
196,abcdeg,bcdeg,abcg,cg,abdefg,acdefg,bcdef,cdg,abdeg,abcdefg
197,af,abcdeg,acdf,bdefg,adefg,abcefg,aef,abcdefg,acdeg,acdefg
198,bcdefg,bcdef,acdefg,abcdefg,befg,abcdef,bcdfg,fg,dfg,abcdg


In [3]:
four_digit_code = original_input.loc[:,11:]
four_digit_code

,11,12,13,14
0,bcd,abcdefg,bcdefg,cdef
1,abcdefg,abcfg,acdefg,abcfg
2,bcefg,abcdefg,afg,acefg
3,abcdg,be,abcdefg,abcdefg
4,acdef,abf,abdefg,abcdfg
...,...,...,...,...
195,bcdef,bceg,bdefg,eg
196,abcdeg,cg,cg,bcdeg
197,acdeg,abcefg,abcdefg,bdefg
198,bcdef,befg,fg,fg


In [4]:
number_of_segments = ten_digits.applymap(len)
number_of_segments

,0,1,2,3,4,5,6,7,8,9
0,6,6,2,5,6,7,4,3,5,5
1,7,2,5,4,3,6,5,6,6,5
2,2,3,6,4,5,7,6,5,6,5
3,6,3,7,5,5,6,5,4,6,2
4,5,2,5,6,3,4,6,5,7,6
...,...,...,...,...,...,...,...,...,...,...
195,7,5,5,3,6,2,4,6,6,5
196,6,5,4,2,6,6,5,3,5,7
197,2,6,4,5,5,6,3,7,5,6
198,6,5,6,7,4,6,5,2,3,5


In [5]:
possible_wirings = pd.DataFrame([['abcdefg']*7]*len(number_of_segments), columns=['a','b','c','d','e','f','g'])
possible_wirings

,a,b,c,d,e,f,g
0,abcdefg,abcdefg,abcdefg,abcdefg,abcdefg,abcdefg,abcdefg
1,abcdefg,abcdefg,abcdefg,abcdefg,abcdefg,abcdefg,abcdefg
2,abcdefg,abcdefg,abcdefg,abcdefg,abcdefg,abcdefg,abcdefg
3,abcdefg,abcdefg,abcdefg,abcdefg,abcdefg,abcdefg,abcdefg
4,abcdefg,abcdefg,abcdefg,abcdefg,abcdefg,abcdefg,abcdefg
...,...,...,...,...,...,...,...
195,abcdefg,abcdefg,abcdefg,abcdefg,abcdefg,abcdefg,abcdefg
196,abcdefg,abcdefg,abcdefg,abcdefg,abcdefg,abcdefg,abcdefg
197,abcdefg,abcdefg,abcdefg,abcdefg,abcdefg,abcdefg,abcdefg
198,abcdefg,abcdefg,abcdefg,abcdefg,abcdefg,abcdefg,abcdefg


# Background info
## Available coding (when wiring is not malfunctioning)


```
  0:      1:      2:      3:      4:
 aaaa    ....    aaaa    aaaa    ....
b    c  .    c  .    c  .    c  b    c
b    c  .    c  .    c  .    c  b    c
 ....    ....    dddd    dddd    dddd
e    f  .    f  e    .  .    f  .    f
e    f  .    f  e    .  .    f  .    f
 gggg    ....    gggg    gggg    ....

  5:      6:      7:      8:      9:
 aaaa    aaaa    aaaa    aaaa    aaaa
b    .  b    .  .    c  b    c  b    c
b    .  b    .  .    c  b    c  b    c
 dddd    dddd    ....    dddd    dddd
.    f  e    f  .    f  e    f  .    f
.    f  e    f  .    f  e    f  .    f
 gggg    gggg    ....    gggg    gggg
 ```


# Part A

## Assignment
the basis is that you try to figure out which columns in `ten_digits` correspond to which digit. Then don't have to know he exact mapping

## Prepare some helper dataframes
One df for each digit, which should yield one `True` per row at the column that codes for that number

In [8]:
df1 = ten_digits.copy()
df1.loc[:,:]= False
df2 = df1.copy()
df3 = df1.copy()
df4 = df1.copy()
df5 = df1.copy()
df6 = df1.copy()
df7 = df1.copy()
df8 = df1.copy()
df9 = df1.copy()
df0 = df1.copy()

In [9]:
def return_only_str_of_segments(df, nth_match=0):
    return df.apply(lambda x: x[~x.isna()].values[nth_match], axis=1)
    
def check_for_uniqueness(df, msg):
    '''A function that checks if the df has only one True value per row and 
    returns the df with strings values'''
    if not (df.sum(axis=1).unique() == np.array([1], dtype=np.int64)):
        raise ValueError(msg)
    return return_only_str_of_segments(ten_digits[df])

### number 1

There is only one that has two segments, so that should match to the number one

In [10]:
row_id = 0
# row = ten_digits.loc[row_id,:]
df1 = number_of_segments == 2
number_one = check_for_uniqueness(df1, "Some rows do not have exactly one string of length 2")
number_one.head()

0    cd
1    df
2    ag
3    be
4    ab
dtype: object

### number 7

There is only one that has three segments, so that should match to the number seven

In [11]:
row_id = 0
# row = ten_digits.loc[row_id,:]
df7 = number_of_segments == 3
number_seven = check_for_uniqueness(df7, "Some rows do not have exactly one string with lenggth 3")
number_seven.head()

0    bcd
1    cdf
2    afg
3    bde
4    abf
dtype: object

### number 4

There is only one that has four segments, so that should match to the number four

In [12]:
row_id = 0
# row = ten_digits.loc[row_id,:]
df4 = number_of_segments == 4
number_four = check_for_uniqueness(df4, "Some rows do not have exactly one string with length 4")
number_four.head()

0    cdef
1    bdef
2    abeg
3    befg
4    abeg
dtype: object

### number 3

There is only one that has five segments, of which 2 of them coincide with the segments of number one. So that should match to the number three

In [59]:
df3 = number_of_segments == 5
number_three = ten_digits[df3]
for row_idx, row_i in number_three.iterrows():
#     row_i
# x = 'df'
# (number_one.loc[row_idx][0] in x) and (number_one.loc[row_idx][1] in x)
    col_idx = row_i.loc[~row_i.isna()].apply(lambda x: (number_one.loc[row_idx][0] in x) and (number_one.loc[row_idx][1] in x))
    df3.loc[row_idx,:]=False
    df3.loc[row_idx, col_idx.loc[col_idx].index] =  True
# number_three = ten_digits[df3]
number_three = check_for_uniqueness(df3, "not unique number three determined")
number_three
# col_idx.loc[col_idx].index

# return_only_str_of_segments(ten_digits[df3], nth_match=pd.IndexSlice[:])
# check_for_uniqueness(df3, "Some rows do not have exactly one string with length 5")


0      bcdeg
1      bcdfg
2      acefg
3      abdeg
4      abdef
       ...  
195    bdefg
196    bcdeg
197    adefg
198    bcdfg
199    bcdef
Length: 200, dtype: object

### Number 5

There is only one number that has 3 segments in common with `number_four` while having a length of `5`. That is `number_five`. 


In [86]:
df5 = (number_of_segments == 5) & (~df3)
for row_idx, row_i in ten_digits[df5].iterrows():
    pass
    col_idx = row_i.loc[~row_i.isna()].apply(lambda x: len(set(x).intersection(set(number_four.loc[row_idx])))==3)
    df5.loc[row_idx,:]=False
    df5.loc[row_idx, col_idx.loc[col_idx].index] =  True
number_five = check_for_uniqueness(df5, "not unique number five determined")
number_five

0      bdefg
1      bcdeg
2      bcefg
3      adefg
4      bdefg
       ...  
195    bcdef
196    abdeg
197    acdeg
198    bcdef
199    cdefg
Length: 200, dtype: object

### Number 2
Number two is the only number with 5 segments which is not number three or five

In [87]:
df2 = (number_of_segments == 5) & (~df3) & (~df5)
number_two = check_for_uniqueness(df2, "2 not unique")
number_two.head()

0    abceg
1    abcfg
2    acdef
3    abcdg
4    acdef
dtype: object

### Number 6
Number six is the only one that has 6 segments of which only 1 coincides with `number_one` 

In [88]:
df6 = (number_of_segments == 6) 
for row_idx, row_i in ten_digits[df6].iterrows():
    col_idx = row_i.loc[~row_i.isna()].apply(lambda x: len(set(x).intersection(set(number_one.loc[row_idx])))==1)
    df6.loc[row_idx,:]=False
    df6.loc[row_idx, col_idx.loc[col_idx].index] =  True
number_six = check_for_uniqueness(df6, "not unique number five determined")
number_six

0      abdefg
1      abcdeg
2      bcdefg
3      acdefg
4      bcdefg
        ...  
195    abcdef
196    abdefg
197    abcdeg
198    abcdef
199    acdefg
Length: 200, dtype: object

### Number 9
Number nine is the only one that has 6 segments of which only 5 coincides with `number_three` 

In [89]:
df9 = (number_of_segments == 6) 
for row_idx, row_i in ten_digits[df9].iterrows():
    col_idx = row_i.loc[~row_i.isna()].apply(lambda x: len(set(x).intersection(set(number_three.loc[row_idx])))==5)
    df9.loc[row_idx,:]=False
    df9.loc[row_idx, col_idx.loc[col_idx].index] =  True
number_nine = check_for_uniqueness(df9, "not unique number five determined")
number_nine

0      bcdefg
1      bcdefg
2      abcefg
3      abdefg
4      abdefg
        ...  
195    bcdefg
196    abcdeg
197    acdefg
198    bcdefg
199    bcdefg
Length: 200, dtype: object

### Number 0
Number zero is the only one that has 6 segments that is not a number 6 or 9

In [90]:
df0 = (number_of_segments == 6) & (~df6) & (~df9)
number_zero = check_for_uniqueness(df0, "not unique number five determined")
number_zero

0      abcdfg
1      acdefg
2      abcdfg
3      abcdef
4      abcdfg
        ...  
195    acdefg
196    acdefg
197    abcefg
198    acdefg
199    abcdfg
Length: 200, dtype: object

###  Number 8
Number eight is the only one with all 7 number_of_segments

In [93]:
df8 = (number_of_segments == 7)
number_eight = check_for_uniqueness(df8, "Number eight nog unique")
number_eight

0      abcdefg
1      abcdefg
2      abcdefg
3      abcdefg
4      abcdefg
        ...   
195    abcdefg
196    abcdefg
197    abcdefg
198    abcdefg
199    abcdefg
Length: 200, dtype: object

## All number are resolved, next step?
Check what answer is required by the test. this can probably be given by the lookup tables.